# Assignment 3

In [1]:
import pandas as pd
import numpy as np
from surprise import KNNWithMeans,Dataset, accuracy,Reader
from surprise.model_selection import train_test_split

In [2]:
ratings = pd.read_csv(r"C:\Users\Administrator\Desktop\machine\Assignment3\ml-latest-small\ratings.csv")
movies = pd.read_csv(r"C:\Users\Administrator\Desktop\machine\Assignment3\ml-latest-small\movies.csv")
tags = pd.read_csv(r"C:\Users\Administrator\Desktop\machine\Assignment3\ml-latest-small\tags.csv")
links = pd.read_csv(r"C:\Users\Administrator\Desktop\machine\Assignment3\ml-latest-small\links.csv")

In [3]:
ratings.shape

(100836, 4)

In [4]:
movies.shape

(9742, 3)

In [5]:
tags.shape

(3683, 4)

In [6]:
links.shape

(9742, 3)

## 1. Create recommender system using ratings.csv file. 

In [7]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [11]:
reader = Reader(rating_scale=(1,5))

data = Dataset.load_from_df(ratings[["userId","movieId","rating"]],reader)

In [12]:
[trainset,testset] = train_test_split(data,test_size=.15,shuffle=True)

recom = KNNWithMeans(k=70, sim_options={"Name":"Cosine","user_based":True})

recom.fit(data.build_full_trainset())

test_pred = recom.test(testset)

RMSE = accuracy.rmse(test_pred)

print("RMSE =",RMSE)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.7172
RMSE = 0.7171845176760988


## 2.Display predicted rating for a particular userID and MovieID combinations (both taken as keyboard input)

In [13]:
userID = int(input("Enter user ID: "))
movieID = int(input("Enter movie ID: "))
print("The rating value for  userID:",userID, "and", "movieID:",movieID, "is :",round((recom.predict(userID,movieID)[3]),2))

Enter user ID: 50
Enter movie ID: 50
The rating value for  userID: 50 and movieID: 50 is : 3.45


## 3. Recommend a user to watch top 10 movies, which has not watched yet (as per his rating predictions).Take userID as a keyboard input. Fix the threshold rating as 2.5

In [17]:
movies_lst = {} 
user_id = int(input("Enter the user id:"))
for movies_id in range(1,193609):
    if recom.predict(user_id,mov_id)[3] > 2.5:
        movies_lst[movies_id] = recom.predict(user_id,movies_id)[3]
        
        
# finding 10 highest values in a Dictionary    
  
from collections import Counter
k = Counter(mov_lst)

high = k.most_common(10)
print("\n")
print("Sugested 10 movies for user {0} are listed below".format(user_id))
for i in range(10):
    print(i+1,"-->",list(movies[movies.movieId==high[i][0]]["title"])[0])

Enter the user id:300


Sugested 10 movies for user 300 are listed below
1 --> On the Beach (1959)
2 --> The Big Bus (1976)
3 --> Galaxy of Terror (Quest) (1981)
4 --> Looker (1981)
5 --> Alien Contamination (1980)
6 --> Master of the Flying Guillotine (Du bi quan wang da po xue di zi) (1975)
7 --> Jetée, La (1962)
8 --> Unfaithfully Yours (1948)
9 --> Match Factory Girl, The (Tulitikkutehtaan tyttö) (1990)
10 --> Strictly Sexual (2008)


## 4.Display the MovieID, IMDB ID, Average User Rating (excluding predictions), genres and tag of all the movies found in Step 3 as a data frame

In [44]:
avg_rating = {}
for i in range(1,193609):
    if (i==44)|(i==46)|(i==48):
        pass
    else:
        avg_rating[i] = round((ratings[ratings.movieId == i]['rating'].mean()),2)

In [45]:
df = pd.merge(ratings[["movieId","rating"]],tags[["movieId","tag"]],how="left",on="movieId")

df2 = pd.merge(df,links[["movieId","imdbId"]],how="left",on="movieId")
df3 = pd.merge(df2,movies[["movieId","genres"]],how="left",on="movieId")
df3.head()

,movieId,rating,tag,imdbId,genres
0,1,4.0,pixar,114709,Adventure|Animation|Children|Comedy|Fantasy
1,1,4.0,pixar,114709,Adventure|Animation|Children|Comedy|Fantasy
2,1,4.0,fun,114709,Adventure|Animation|Children|Comedy|Fantasy
3,3,4.0,moldy,113228,Comedy|Romance
4,3,4.0,old,113228,Comedy|Romance


In [46]:
ls = []
for i in range(10):
    ls.append(high[i][0])
df4 = df3[df3["movieId"].isin(ls)]


df4 = df4.drop("rating",axis=1)

In [47]:
id_ratings_pair = [(x,y) for x,y in avg_rating.items() if x in ls]

new_ratings = []
for i in range(len(id_ratings_pair)):
    new_ratings.append(id_ratings_pair[i][1])


In [48]:
new_ratings

[4.5, 5.0, 5.0, 4.5, 5.0, 4.5, 4.5, 5.0, 5.0, 5.0]